In [1]:
import pandas as pd
import numpy as np
import requests
import grequests
import os.path
import tensorflow as tf

import datetime

In [2]:
# shamelessly taken from https://danijar.com/variable-sequence-lengths-in-tensorflow/
def getSequenceLengths(batchOfSequences):
  used = tf.sign(tf.reduce_max(tf.abs(batchOfSequences), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [ ]:
import argparse

parser = argparse.ArgumentParser(description = "Please insert the train flag")
parser.add_argument('-t', '--train', action = "store",
                    help='If true, we train and save. Else, otherwise.', required = True)

my_args = vars(parser.parse_args())
trainFlag = my_args['train']
trainFlag = trainFlag.lower() in ("True", "t", "true", "1", 1)

In [3]:
logging = tf.logging

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
print(datetime.datetime.now())

2017-08-26 21:56:33.046505


In [6]:
df = np.load('invokerItems.npy')
match_ids = pd.read_pickle('final_processed_invoker_data')['matchId']

x = np.random.rand(df.shape[0])
mask = np.where(x < 0.75)[0]
mask2 = np.where(x >= 0.75)[0]
df_train = df[mask, :]
df_test = df[mask2, :]
match_ids_train = match_ids.iloc[mask]
match_ids_test = match_ids.iloc[mask2]

In [7]:
# 265 is highest ID, and we need a zero for empty
NUM_ITEMS = 266
DIM = NUM_ITEMS

batchSize = int(len(df) * 1.0 /20)
batched_ItemData = tf.placeholder(tf.float32, shape=[None, None, DIM])

batched_ItemData = tf.pad(batched_ItemData, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
# shift each sequence to the left by one (stride to the right by 1)
# and then pad at the end to maintain same lengths!
batched_ItemData_strided = tf.strided_slice(batched_ItemData, [0, 1, 0], tf.shape(batched_ItemData))
targets = tf.pad(batched_ItemData_strided, [[0, 0], [0, 1], [0, 0]], "CONSTANT")
MAX_SequenceLength = df.shape[1]
batched_ItemDataSequenceLengths = getSequenceLengths(batched_ItemData)
lstmHiddenSize = 50
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmHiddenSize)

In [9]:
state = tf.Variable(lstmCell.zero_state(batchSize, tf.float32), trainable=False, name='lstm_state')

lstmOutput, last_states = tf.nn.dynamic_rnn(
    cell=lstmCell,
    dtype=tf.float32,
    sequence_length=batched_ItemDataSequenceLengths,
    inputs=batched_ItemData,
    initial_state = tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]))

with tf.control_dependencies([state.assign(last_states)]):
    lstmOutput = tf.identity(lstmOutput)

In [10]:
lstmOutput = tf.reshape(lstmOutput, [-1, lstmHiddenSize])

In [12]:
with tf.variable_scope('LayerToLogits'):
    weights = tf.Variable(tf.random_normal([lstmHiddenSize, NUM_ITEMS], stddev = 1.0/NUM_ITEMS/100), name='weights_1')
    bias = tf.Variable(tf.random_normal([NUM_ITEMS], stddev = 1.0/NUM_ITEMS/100), name='bias_1')

In [13]:
logits = tf.matmul(lstmOutput, weights) + bias
targets = tf.reshape(targets, [-1, DIM])
mask = tf.sign(tf.reduce_max(tf.abs(targets), 1))
loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = logits)
loss = mask * loss
loss = tf.reshape(loss, shape=[batchSize, MAX_SequenceLength])
# loss is 2d in a dynamic length way and then make it 1d to then reduce on
mean_loss = tf.reduce_sum(loss, reduction_indices=1) / tf.cast(batched_ItemDataSequenceLengths, tf.float32)
cost = tf.reduce_mean(mean_loss)

In [14]:
# default of 0.001
learning_rate = 0.001
# default of 0.99
beta1 = 0.99
# default of 0.999
beta2 = 0.999
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1, beta2 = beta2)

gradients, variables = zip(*optimizer.compute_gradients(cost))
gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
train_op = optimizer.apply_gradients(zip(gradients, variables))

** Generate summary information **

In [16]:
# code from tensorflow documentation
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.variable_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.variable_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
    
with tf.variable_scope('weights'):
    variable_summaries(weights)
    
with tf.variable_scope('bias'):
    variable_summaries(bias)

with tf.variable_scope("loss"):
    variable_summaries(mean_loss)
    # this number is already calculated in variable_summaries of mean_loss but be explicit
    tf.summary.scalar('cost', cost)

summary_op = tf.summary.merge_all()

In [17]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
ckpoint_dir = os.path.join(os.getcwd(), 'model-backups/model.ckpt')

In [20]:
def train(sess):
    numEpochs = 1000
    numBatches = 1000
    for epochIter in range(numEpochs):
        print('Epoch: {0}'.format(epochIter))
        for batchItr in range(numBatches):
            samp = np.random.choice(np.arange(df_train.shape[0]), size=batchSize, replace=True, p=None)
            if (epochIter+1) % 50 == 0:
                summaries, _ = sess.run([summary_op, train_op], 
                                         feed_dict = {batched_ItemData : df_train[samp, :, :]})
                train_writer.add_summary(summaries, epochIter)
                saver.save(sess, ckpoint_dir)
            else:
                sess.run(train_op, feed_dict = {batched_ItemData : df_train[samp, :, :]})

In [21]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('logs/train',
                                      sess.graph)
    test_writer = tf.summary.FileWriter('logs/test')
    if (trainFlag):
        sess.run(init)
        train(sess)
    else:
        print('Doing test')
        saver.restore(sess, ckpoint_dir)

Epoch: 0


KeyboardInterrupt: 